In [144]:
### all of the imports go into requirements!!!

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import pathlib
from pathlib import Path
import string
from datetime import timedelta
from nltk.corpus import stopwords
from nltk import word_tokenize
!pip install nltk
from collections import Counter

In [145]:
# Load your existing database into a DataFrame
data = pd.read_csv('/home/yannik/ticket-control-bvg/data/telegram_data.csv')  # Replace with the path to your database file
# Notice the .copy() to copy the values 
data = data.copy()

In [146]:
data.head(10)

group       sender   text                 date
0  freifahren_BE          NaN    NaN  2018-02-15 11:01:00
1  freifahren_BE  283284722.0     Hä  2018-02-15 11:14:10
2  freifahren_BE  462810467.0    NaN  2018-02-15 11:17:08
3  freifahren_BE  462810467.0    NaN  2018-02-15 11:18:17
4  freifahren_BE  462810467.0    NaN  2018-02-15 11:21:17
5  freifahren_BE  277514659.0    NaN  2018-02-15 11:26:31
6  freifahren_BE   21995695.0    YES  2018-02-15 11:26:54
7  freifahren_BE   21995695.0    NaN  2018-02-15 11:26:57
8  freifahren_BE   21995695.0  😂😂  😂  2018-02-15 11:27:08
9  freifahren_BE  462810467.0    NaN  2018-02-15 11:28:09

In [147]:
data['sender'] = data['sender'].astype(str)

In [148]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251280 entries, 0 to 251279
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   group   251280 non-null  object
 1   sender  251280 non-null  object
 2   text    206853 non-null  object
 3   date    251280 non-null  object
dtypes: object(4)
memory usage: 7.7+ MB


## Cleaning the dataset

In [149]:
len(data)

251280

### Date > Datetime Format

In [150]:
data["date"] = pd.to_datetime(data["date"])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251280 entries, 0 to 251279
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   group   251280 non-null  object        
 1   sender  251280 non-null  object        
 2   text    206853 non-null  object        
 3   date    251280 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 7.7+ MB


### Clean dataframe of na/nan,....

In [151]:
data = data[data['text'].notna()]

In [152]:
data['text'] = data['text'].str.strip()

In [153]:
data['text'].replace('', np.nan, inplace=True)

In [154]:
data.dropna(subset=['text'], inplace=True)

In [155]:
len(data)

206853

### Merging rows together that belong to the same sender within a given timeframe

In [156]:
df = data.sort_values(by=['sender', 'date'])

df['time_diff'] = df.groupby('sender')['date'].diff()

data_clean = df.groupby(['sender', (df['time_diff'] > timedelta(minutes=10)).cumsum()]).agg({'text': ' '.join, 
                                                                                         'date': 'first'}).reset_index()
data_clean

sender  time_diff  \
0       -1001345643242.0          0   
1       -1001345643242.0          1   
2       -1001570966831.0          1   
3       -1001570966831.0          2   
4       -1001570966831.0          3   
...                  ...        ...   
147952       999684523.0     127404   
147953       999684523.0     127405   
147954       999684523.0     127406   
147955       999708285.0     127406   
147956       999708285.0     127407   

                                                     text                date  
0       Kann ich beim Kauf eines Tickets 9 € in alle S... 2022-06-01 21:42:14  
1                                                Ok danke 2022-06-01 21:58:49  
2       Kontis Mehringdamm Weiß ich nich bin schon Weg... 2022-03-03 16:41:36  
3                                      Kontis Moritzplatz 2022-03-10 14:38:26  
4       Ganz viele kontis am kotti stehen vor der Bahn... 2022-03-21 08:53:11  
...                                                   ...                 ...  
147952          U2 bei RosaLixenburger Platz. . 3 manner. 2021-03-08 13:48:04  
147953                          U5 weberwiese. 3-4 manner 2021-03-08 14:52:48  
147954            U2 Rosa Luxemburg Platz. Richtung Alex. 2021-04-20 11:53:17  
147955  Wurde mit 10 Uhr ticket um 9 Uhr am südstern  ... 2020-06-19 07:45:47  
147956  Wurde mit 10 Uhr ticket um 9 Uhr am südstern  ... 2020-06-19 18:18:52  

[147957 rows x 4 columns]

In [157]:
data_clean['sender'] = data_clean['sender'].astype(str)

In [158]:
data_clean.tail(30)

sender  time_diff  \
147927  999204625.0     127386   
147928  999204625.0     127387   
147929  999204625.0     127388   
147930  999204625.0     127389   
147931  999204625.0     127390   
147932  999204625.0     127391   
147933  999215014.0     127391   
147934  999215014.0     127392   
147935  999215014.0     127393   
147936  999215014.0     127394   
147937  999215014.0     127395   
147938  999365610.0     127395   
147939  999365610.0     127396   
147940  999365610.0     127397   
147941  999365610.0     127398   
147942  999365610.0     127399   
147943  999416326.0     127399   
147944  999425588.0     127399   
147945  999425588.0     127400   
147946  999425588.0     127401   
147947  999425588.0     127402   
147948  999425588.0     127403   
147949  999489207.0     127403   
147950  999489207.0     127404   
147951  999510343.0     127404   
147952  999684523.0     127404   
147953  999684523.0     127405   
147954  999684523.0     127406   
147955  999708285.0     127406   
147956  999708285.0     127407   

                                                     text                date  
147927  S1 nach Wannsee Höhe julius leber. 2 Frauen, s... 2020-10-22 05:57:18  
147928  S1 Friedrichstraße auf dem unteren Bahnhof. Si... 2020-12-22 10:21:47  
147929  2 ppl Friedrichstraße - 2nd one all black clot... 2021-02-25 09:24:20  
147930  2 Männer eine Frau Kaiser Wilhelm Platz gerade... 2021-05-12 11:33:47  
147931  Friedrichstraße direction west in s5 Direction... 2021-11-27 22:31:52  
147932   S1 at Friedrichstraße direction north. White bag 2021-12-08 07:40:29  
147933                               S7 linie sind welche 2020-08-06 10:52:07  
147934  hackeschee markt 3 menschen Jetzt Friedrichstr... 2020-08-06 11:22:34  
147935  2 kontros u 6 kochstraße ausgestiegen 2 kontro... 2020-08-12 06:11:57  
147936         polizei rathaus steglitz mit 1 kontrolleur 2020-08-20 15:44:48  
147937  s41 tempelhof und s41 ostkreuz \nfett kontros ... 2020-11-07 14:56:05  
147938                                        Immer noch… 2023-03-24 12:45:03  
147939  BVG fährt alles, nur S-Bahn & co werden bestreikt 2023-03-27 09:27:54  
147940  Sind drin, hinteres Ende des Zuges. Grade Mehr... 2023-04-11 12:18:39  
147941  Nicht wirklich frei fahren relevant, aber viel... 2023-04-13 14:53:35  
147942  2x Ordnungsamt, vorderer Teil des Zuges am Bun... 2023-05-17 09:43:12  
147943                            U6 Platz der Luftbrücke 2023-07-05 17:13:00  
147944  Hallo hätte jemand ein Berlin Ticket u vom ges... 2020-08-17 08:03:02  
147945  Yes got one yesterday at 13:30 would be lovely... 2020-08-17 08:20:29  
147946  Hallo hätte jemand ein Berlin Ticket u vom ges... 2020-08-17 10:48:37  
147947  U 6 keizer augastina black clothes mawhawk and... 2021-01-15 13:07:43  
147948                                        U7 Südstern 2021-02-04 13:32:56  
147949  Habe ein blanko Berlin-Pass Ticket S ab zu geb... 2020-06-28 11:19:15  
147950  Ich habe noch ein blanko Berlin Ticket S ( für... 2020-06-30 18:26:45  
147951                       2 guys 2 girls U5 weberwiese 2020-06-26 08:22:52  
147952          U2 bei RosaLixenburger Platz. . 3 manner. 2021-03-08 13:48:04  
147953                          U5 weberwiese. 3-4 manner 2021-03-08 14:52:48  
147954            U2 Rosa Luxemburg Platz. Richtung Alex. 2021-04-20 11:53:17  
147955  Wurde mit 10 Uhr ticket um 9 Uhr am südstern  ... 2020-06-19 07:45:47  
147956  Wurde mit 10 Uhr ticket um 9 Uhr am südstern  ... 2020-06-19 18:18:52

In [159]:
len(data_clean)

147957

### De-Emojify

In [160]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [161]:
data_clean['text']= data_clean['text'].apply(lambda x: remove_emojis(str(x)))

In [162]:
data_clean.head()

sender  time_diff  \
0  -1001345643242.0          0   
1  -1001345643242.0          1   
2  -1001570966831.0          1   
3  -1001570966831.0          2   
4  -1001570966831.0          3   

                                                text                date  
0  Kann ich beim Kauf eines Tickets 9 € in alle S... 2022-06-01 21:42:14  
1                                           Ok danke 2022-06-01 21:58:49  
2  Kontis Mehringdamm Weiß ich nich bin schon Weg... 2022-03-03 16:41:36  
3                                 Kontis Moritzplatz 2022-03-10 14:38:26  
4  Ganz viele kontis am kotti stehen vor der Bahn... 2022-03-21 08:53:11

### Drop rows where text column was empty/had only spaces

In [163]:
data_clean = data_clean[data_clean['text'].notna()]

In [164]:
len(data_clean)

147957

In [165]:
data_clean['text'] = data_clean['text'].str.strip()

In [166]:
len(data_clean)

147957

In [167]:
data_clean['text'].replace('', np.nan, inplace=True)

In [168]:
len(data_clean)

147957

In [169]:
data_clean.dropna(subset=['text'], inplace=True)

In [170]:
len(data_clean)

147472

### Drop duplicates

In [171]:
duplicate_count = data_clean.duplicated().sum()
duplicate_count ### This is a lot of duplicates

0

In [172]:
### dropping duplicates
data_clean = data_clean.drop_duplicates()

In [173]:
len(data_clean)

147472

### Lowercasing all strings

In [174]:
### lowercasing all strings
data_clean['text']= data_clean['text'].apply(lambda x: x.lower())

### identifying and removing stopwords

In [175]:
### generating list of default stop words
stop_words = set(stopwords.words('german'))

In [176]:
len(stop_words)

232

In [177]:
# Add multiple words using the update method
new_words_to_add = ['männlich', 'weiblich', 'gelesen','weste','westen', 'shirt', 'pulli', 'jacke', 'jacken', 'ticket', 'tickets', 'eingestiegen', 'ausgestiegen', 'steigen', 'schwarze', 'schwarz', 'männer', 'haare', 'the', 'stehen', 'gelesene', 'blaue', 'with']
stop_words.update(new_words_to_add)

In [178]:
# Remove unwanted stopwords
my_wanted_words = ['nach', 'bei', 'von', 'vom' 'zum', 'über', 'bis']
final_stopwords = stop_words - set(my_wanted_words)

In [179]:
def stopword(text):
    word_tokens = word_tokenize(text)
    text = [w for w in word_tokens if not w in final_stopwords]  ## if w isn't in final_stopwords, return w 
    return ' '.join(text)  ##transforming list into string again

In [180]:
data_clean['text'] = data_clean['text'].apply(stopword)
data_clean['text']

0                             beim kauf 9 € städte reisen ?
1                                                  ok danke
2         kontis mehringdamm weiß nich schon sorry : / g...
3                                        kontis moritzplatz
4                   ganz viele kontis kotti bahn rum blauen
                                ...                        
147952          u2 bei rosalixenburger platz . . 3 manner .
147953                           u5 weberwiese . 3-4 manner
147954            u2 rosa luxemburg platz . richtung alex .
147955    wurde 10 uhr 9 uhr südstern kontrolliert . zuf...
147956    wurde 10 uhr 9 uhr südstern kontrolliert . zuf...
Name: text, Length: 147472, dtype: object

In [181]:
data_clean.head()

sender  time_diff  \
0  -1001345643242.0          0   
1  -1001345643242.0          1   
2  -1001570966831.0          1   
3  -1001570966831.0          2   
4  -1001570966831.0          3   

                                                text                date  
0                      beim kauf 9 € städte reisen ? 2022-06-01 21:42:14  
1                                           ok danke 2022-06-01 21:58:49  
2  kontis mehringdamm weiß nich schon sorry : / g... 2022-03-03 16:41:36  
3                                 kontis moritzplatz 2022-03-10 14:38:26  
4            ganz viele kontis kotti bahn rum blauen 2022-03-21 08:53:11

### Remove punctuation

In [182]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [183]:
for element in string.punctuation:
    data_clean['text'] = data_clean['text'].str.replace(element, '')

In [184]:
data_clean['text'] = data_clean['text'].replace('', np.nan)

In [185]:
data_clean.head()

sender  time_diff  \
0  -1001345643242.0          0   
1  -1001345643242.0          1   
2  -1001570966831.0          1   
3  -1001570966831.0          2   
4  -1001570966831.0          3   

                                                text                date  
0                       beim kauf 9 € städte reisen  2022-06-01 21:42:14  
1                                           ok danke 2022-06-01 21:58:49  
2  kontis mehringdamm weiß nich schon sorry   gle... 2022-03-03 16:41:36  
3                                 kontis moritzplatz 2022-03-10 14:38:26  
4            ganz viele kontis kotti bahn rum blauen 2022-03-21 08:53:11

In [186]:
data_clean['text'] = data_clean['text'].str.strip()

In [187]:
data_clean.dropna(subset='text', inplace=True)

In [188]:
len(data_clean)

147039

In [189]:
data_clean = data_clean.drop_duplicates(subset='text')

In [190]:
data_clean = data_clean[data_clean['text'] != '']

In [191]:
data_clean.dropna(subset='text', inplace=True)

In [192]:
len(data_clean)

137656

In [193]:
data_clean['text'] = data_clean['text'].str.replace('ß', 'ss')
data_clean['text'] = data_clean['text'].str.replace('ä', 'ae')
data_clean['text'] = data_clean['text'].str.replace('ö', 'oe')
data_clean['text'] = data_clean['text'].str.replace('ü', 'ue')

In [194]:
data_clean = data_clean.sort_values(by=['date', 'sender'])

In [195]:
# Concatenate all sentences into a single string
all_sentences = ' '.join(data_clean['text'])

# Tokenize the concatenated string into words
words = all_sentences.split()

# Count the occurrences of each word
word_counts = Counter(words)

# Sort the words by their occurrences in descending order
sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

# Print the top 100 words
for word, count in sorted_words[:100]:
    print(f'{word}: {count}')

richtung: 44773
2: 27904
u8: 16983
gerade: 14394
3: 12134
1: 11493
u7: 9610
s: 9240
zwei: 9197
u6: 8484
raus: 8131
platz: 6976
kontrolle: 6837
u9: 6554
u: 6427
str: 6070
gleis: 6026
maenner: 5743
nach: 5654
bahnsteig: 5235
s42: 5211
kontrolleure: 5180
s41: 5154
strasse: 5097
allee: 5073
m: 4995
u5: 4836
tor: 4686
4: 4665
personen: 4553
frau: 4469
u2: 4386
von: 4366
gleich: 4162
alex: 4055
spandau: 4051
warschauer: 3941
beide: 3924
wittenau: 3801
blauwesten: 3695
person: 3682
typen: 3669
cap: 3605
jeans: 3594
bahn: 3569
bvg: 3381
mann: 3370
rathaus: 3363
s7: 3321
schwarzer: 3321
tempelhof: 3310
frankfurter: 3304
bei: 3268
at: 3185
and: 2897
grad: 2871
leute: 2760
hermannplatz: 2760
osloer: 2733
blauen: 2712
s1: 2711
hermannstrasse: 2701
grade: 2689
u1: 2685
bitte: 2674
alt: 2666
alexanderplatz: 2615
kontis: 2508
m10: 2504
hermannstr: 2468
to: 2466
rudow: 2460
bahnhof: 2446
ostkreuz: 2445
1x: 2439
friedrichstrasse: 2399
neukoelln: 2381
drei: 2359
ringbahn: 2339
a: 2325
gekleidet: 2320
ko

In [196]:
data_clean.head()

sender  time_diff  \
56130  283284722.0      47821   
45427   21995695.0      38280   
84475  462810467.0      73226   
51061  259910222.0      43255   
87661  489941084.0      75998   

                                                  text                date  
56130                                              hae 2018-02-15 11:14:10  
45427                             yes aehnliches schon 2018-02-15 11:26:54  
84475                                   kanns losgehen 2018-02-15 11:29:46  
51061  fahr eigentlich immer schwerbehinderten ausweis 2018-02-15 11:31:44  
87661                              danke fur einladung 2018-02-15 11:54:57

In [197]:
data_handover = data_clean.drop('time_diff', axis=1)
data_handover.head(100)

sender                                               text  \
56130  283284722.0                                                hae   
45427   21995695.0                               yes aehnliches schon   
84475  462810467.0                                     kanns losgehen   
51061  259910222.0    fahr eigentlich immer schwerbehinderten ausweis   
87661  489941084.0                                danke fur einladung   
...            ...                                                ...   
75869  409237588.0                                       normal kotti   
45596  220412550.0       mehr  verteilen halt gleichmaessig tuerhoehe   
93324  523929370.0  u1 richtung warschauer gleich goerlitzer bhf 2...   
39565  195727113.0                         grosskontrolle moritzplatz   
59475  302346794.0  s 3 nach erkner 2 kontrollettis grad bei wuhlh...   

                     date  
56130 2018-02-15 11:14:10  
45427 2018-02-15 11:26:54  
84475 2018-02-15 11:29:46  
51061 2018-02-15 11:31:44  
87661 2018-02-15 11:54:57  
...                   ...  
75869 2018-02-22 20:35:37  
45596 2018-02-22 20:36:33  
93324 2018-02-23 08:42:50  
39565 2018-02-23 11:07:07  
59475 2018-02-23 12:22:12  

[100 rows x 3 columns]

In [198]:
data_handover.to_csv('data_handover_for_team.csv')

In [199]:
data_handover.head()

sender                                             text  \
56130  283284722.0                                              hae   
45427   21995695.0                             yes aehnliches schon   
84475  462810467.0                                   kanns losgehen   
51061  259910222.0  fahr eigentlich immer schwerbehinderten ausweis   
87661  489941084.0                              danke fur einladung   

                     date  
56130 2018-02-15 11:14:10  
45427 2018-02-15 11:26:54  
84475 2018-02-15 11:29:46  
51061 2018-02-15 11:31:44  
87661 2018-02-15 11:54:57